In [1]:
! du -sh ./*

52K	./LSTM.ipynb
3.5M	./data_analysis.ipynb
548K	./data_analysis_jingzhi.ipynb
40K	./feature_engineering.ipynb
4.0K	./notebook_template.ipynb
184K	./save_test.npz
236K	./test.ipynb
32K	./train-Copy1.ipynb
104K	./train.ipynb


In [1]:
import sys
import os
from time import time 
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import multiprocessing



sys.path.append('../')
from utils import (
    check_columns, 
    check_category_column, 
    save_model, 
    transform_category_column,
    get_time_diff,
    load_model,
    LogManager,
    timer,
    standard_scale,
    log_scale,
    remove_cont_cols_with_small_std,
)
import conf
from mlpipeline.feature_engineering import (
    USING_LABEL,
    FAULT_LABEL,
)

In [2]:
df = pd.read_feather(os.path.join(conf.DATA_DIR, "test_file_03.feather"))

In [3]:
test_dict = dict(tuple(df.groupby(["serial_number", "model"])))

In [4]:
fault_df = df[df["flag"]==1]

In [5]:
fault_dic = dict(tuple(fault_df.groupby(["serial_number", "model"])))

In [6]:
fault_dic.keys()

dict_keys([('disk_103990', 1), ('disk_104617', 1), ('disk_108276', 1), ('disk_110185', 1), ('disk_113535', 1), ('disk_11374', 1), ('disk_116937', 1), ('disk_117550', 2), ('disk_11788', 2), ('disk_119072', 1), ('disk_123077', 2), ('disk_123598', 1), ('disk_125746', 1), ('disk_125953', 2), ('disk_126915', 2), ('disk_127609', 1), ('disk_128125', 2), ('disk_128986', 2), ('disk_131630', 2), ('disk_133408', 2), ('disk_13400', 1), ('disk_13593', 1), ('disk_135970', 1), ('disk_138123', 1), ('disk_138774', 1), ('disk_139345', 1), ('disk_139874', 1), ('disk_139880', 1), ('disk_141029', 2), ('disk_142247', 1), ('disk_143767', 1), ('disk_145360', 1), ('disk_15301', 2), ('disk_15938', 2), ('disk_19368', 1), ('disk_21342', 1), ('disk_21882', 1), ('disk_23869', 1), ('disk_2510', 2), ('disk_28293', 1), ('disk_30245', 1), ('disk_3306', 2), ('disk_3515', 2), ('disk_38051', 1), ('disk_4086', 1), ('disk_41015', 1), ('disk_45690', 2), ('disk_46746', 2), ('disk_47904', 2), ('disk_48529', 2), ('disk_49239', 

In [7]:
instance = test_dict[('disk_100420', 1)].set_index("dt")
instance

,serial_number,model,smart_1_normalized_min_7,smart_3_normalized_min_7,smart_7_normalized_min_7,smart_184_normalized_min_7,smart_187_normalized_min_7,smart_191_normalized_min_7,smart_195_normalized_min_7,smart_5raw_min_7,...,smart_191_normalized,smart_195_normalized,smart_5raw,smart_194raw,smart_197raw,smart_198raw,smart_199raw,model_type,tag,flag
dt,,,,,,,,,,,,,,,,,,,,,
2018-03-01,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-02,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-03,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-04,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-05,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-06,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-07,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,29.0,0.0,0.0,0.0,0,0,0
2018-03-08,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0
2018-03-09,disk_100420,1,83.0,96.0,93.0,100.0,100.0,100.0,23.0,0.0,...,100.0,23.0,0.0,30.0,0.0,0.0,0.0,0,0,0


In [4]:
cols = ['serial_number','model','smart_1_normalized','smart_1raw','smart_3_normalized',
 'smart_3raw','smart_4_normalized','smart_4raw','smart_5_normalized','smart_5raw',
 'smart_7_normalized','smart_7raw','smart_9_normalized','smart_9raw','smart_10_normalized',
 'smart_10raw','smart_12_normalized','smart_12raw','smart_184_normalized','smart_184raw',
 'smart_187_normalized','smart_187raw','smart_188_normalized','smart_188raw','smart_189_normalized',
 'smart_189raw','smart_190_normalized','smart_190raw','smart_191_normalized','smart_191raw',
 'smart_192_normalized','smart_192raw','smart_193_normalized','smart_193raw','smart_194_normalized',
 'smart_194raw','smart_195_normalized','smart_195raw','smart_197_normalized','smart_197raw',
 'smart_198_normalized','smart_198raw','smart_199_normalized','smart_199raw','dt','tag']

In [5]:
df = pd.read_hdf(os.path.join(conf.DATA_DIR, "data_2018_semifinals_all.h5"), columns = cols)

In [6]:
print(df.shape)
df.head()

(36854339, 46)


,serial_number,model,smart_1_normalized,smart_1raw,smart_3_normalized,smart_3raw,smart_4_normalized,smart_4raw,smart_5_normalized,smart_5raw,...,smart_195_normalized,smart_195raw,smart_197_normalized,smart_197raw,smart_198_normalized,smart_198raw,smart_199_normalized,smart_199raw,dt,tag
1111566,disk_21087,1,80.0,104629728.0,96.0,0.0,100.0,23.0,100.0,0.0,...,63.0,104629728.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-01-01,0
3427638,disk_69256,2,84.0,232161968.0,86.0,0.0,100.0,104.0,100.0,0.0,...,15.0,232161968.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-01-01,0
1457138,disk_41304,1,81.0,150906496.0,96.0,0.0,100.0,9.0,100.0,0.0,...,24.0,150906496.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-01-01,0
2913767,disk_73081,2,83.0,189932672.0,88.0,0.0,100.0,12.0,100.0,0.0,...,12.0,189932672.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-01-01,0
2473882,disk_9590,1,68.0,6786815.0,97.0,0.0,100.0,6.0,100.0,0.0,...,12.0,6786815.0,100.0,0.0,100.0,0.0,200.0,0.0,2018-01-01,0


In [7]:
mask = df["dt"] < "20180801"
df = df[mask]
df = df.fillna(method="ffill")
print(df.isnull().sum())

serial_number           0
model                   0
smart_1_normalized      0
smart_1raw              0
smart_3_normalized      0
smart_3raw              0
smart_4_normalized      0
smart_4raw              0
smart_5_normalized      0
smart_5raw              0
smart_7_normalized      0
smart_7raw              0
smart_9_normalized      0
smart_9raw              0
smart_10_normalized     0
smart_10raw             0
smart_12_normalized     0
smart_12raw             0
smart_184_normalized    0
smart_184raw            0
smart_187_normalized    0
smart_187raw            0
smart_188_normalized    0
smart_188raw            0
smart_189_normalized    0
smart_189raw            0
smart_190_normalized    0
smart_190raw            0
smart_191_normalized    0
smart_191raw            0
smart_192_normalized    0
smart_192raw            0
smart_193_normalized    0
smart_193raw            0
smart_194_normalized    0
smart_194raw            0
smart_195_normalized    0
smart_195raw            0
smart_197_no

In [8]:
fault_df = df[df["tag"]!=0]

In [9]:
fault_dict = dict(tuple(fault_df.groupby(["serial_number", "model"])))
del fault_df
data_dict = dict(tuple(df.groupby(["serial_number", "model"])))
del df
gc.collect()

0

In [10]:
print("number of fault disks:", len(fault_dict.keys()))
print("number of normal disks:", len(data_dict.keys())-len(fault_dict.keys()))
print("ratio", len(fault_dict.keys())/(len(data_dict.keys())-len(fault_dict.keys())))

number of fault disks: 964
number of normal disks: 169619
ratio 0.005683325570838172


In [7]:
def generate_time_series_data(instance, seq_len=20):
    data = []
    tags = []
    
    instance = instance.set_index("dt")
        
    num_of_data = len(instance.index) - (seq_len - 1)

    for i in range(num_of_data):
    
        start_date = instance.index[i]
        end_date = instance.index[i+seq_len-1]
        mask = instance.index >= start_date
        mask &= instance.index <= end_date
    
        tag = instance[mask]["tag"][-1]
    
        time_seq_data = np.array(instance[mask].drop(["serial_number", "model", "tag", "flag"], axis=1))
    
        data.append(time_seq_data)
        tags.append(tag)
    
    return (data, tags)

def sample_by_disk(test_dict, fault_dict):
    sampled_keys = []
    ratio=len(fault_dic)/len(test_dict)
    for key in test_dict.keys():
        if key not in fault_dict:
            if np.random.random_sample() > ratio:
                continue
        sampled_keys += [key]
    return sampled_keys

In [8]:
len(sample_by_disk(test_dict, fault_dic))

201

In [9]:
keys = test_dict.keys()
chunk_size = 1e4
with multiprocessing.Pool(processes=8) as p:
    with tqdm(total=len(keys)) as pbar:
        data_ = []
        tag_ = []
        count = 0
        for result in p.imap_unordered(generate_time_series_data, [test_dict[key] for key in keys]):
            data_ += result[0]
            tag_ += result[1]
            pbar.update()
            count += 1
            if count >= chunk_size:                   
                np.savez('../data/save_test_%d.npz' % (count//chunk_size), data = data_, tag = tag_)
                count = 0
                data_ = []    
                tag_ = [] 

  8%|▊         | 11936/151178 [01:47<20:51, 111.22it/s]  Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:

Traceback (most recent call last):


KeyboardInterrupt: 

In [3]:
npzfile = np.load("../data/save_test_1.npz", allow_pickle=True)

In [11]:
print(npzfile["tag"].shape)
npzfile["data"].shape

(116923,)


(116923, 20, 86)

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LSTMModel(nn.Module):
    ''' Encodes time-series sequence '''

    def __init__(self, input_size, hidden_size, num_layers, output_size, drop_out=0.5):      
        '''
        : param input_size:     the number of features in the input X
        : param hidden_size:    the number of features in the hidden state h
        : param num_layers:     number of recurrent layers (i.e., 2 means there are
        :                       2 stacked LSTMs)
        '''
        super(LSTMModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, dropout = drop_out)
        self.classifier = nn.Linear(hidden_size, output_size)
        self.init_weights()

    def forward(self, x_input):
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence;
        :                              hidden gives the hidden state and cell state for the last
        :                              element in the sequence 
        '''
        lstm_out, _ = self.lstm(x_input)
        logit = self.classifier(lstm_out[-1,:,:])

        return logit   

    def init_weights(self):
        '''
        initialize the weight of fc layer.
        '''
        initrange = 0.1
        self.classifier.weight.data.uniform_(-initrange, initrange)
        self.classifier.bias.data.zero_()

    def init_hidden(self, batch_size):
        '''
        initialize hidden state
        : param batch_size:    x_input.shape[1]
        : return:              zeroed hidden state and cell state 
        '''
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))

In [99]:
import time
from sklearn import metrics

def train(model, criterion, optimizer, train_set, epoch):
    model.train()
    total_loss = 0
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)
    model.to(device) 
    criterion.to(device)

    start_time = time.time()
    for i, (inputs, labels) in tqdm(enumerate(train_set)):
        inputs = inputs.transpose(1, 0)
        inputs, labels = inputs.to(device), labels.to(device, dtype=torch.int64)
        model.zero_grad()
        outputs = model(inputs)   
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if i % 10 == 0:
            true = labels.data.cpu()
            pred = torch.max(outputs.data, 1)[1].cpu()
            train_acc = metrics.accuracy_score(true, pred) 
            print(train_acc)
    end_time = time.time()

    print("Epoch: {0:>6}, Train Loss: {1:>6.4}, Train Acc: {2:>6.2%}, Time per Epoch: {5:>6.2}s".format(
            epoch, total_loss / len(train_set), train_acc, end_time - start_time))

def evaluate(model, criterion, val_set):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    total_loss = 0
    pred = []
    label = []
    with torch.no_grad():
        for _, (inputs, labels) in enumerate(val_set): 
            inputs = inputs.transpose(1, 0)
            inputs, labels = inputs.to(device), labels.to(device)
            output = model(inputs)
            loss = criterion(output, labels)
            total_loss += loss.item()

            label.extend(labels.data.cpu().numpy())
            pred.extend(torch.max(output.data, 1)[1].cpu())
            
    val_acc = metrics.accuracy_score(label, pred)
    val_loss = total_loss / len(val_set)
    print("Val Loss: {1:>6.4}, Val Acc: {2:>6.2%}".format(val_loss, val_acc))
    return val_acc, val_loss


In [10]:
data = npzfile["data"]
target = npzfile["tag"]

In [43]:
target.reshape(-1,1)
data = np.array(data, dtype=np.float32)
data[np.isnan(data)]=0

In [76]:
train_size = int(0.8 * len(target))
test_size = len(target) - train_size

dataset = torch.utils.data.TensorDataset(torch.Tensor(data), torch.Tensor(target))
train_data, val_data = torch.utils.data.random_split(dataset, [train_size, test_size])

In [77]:
train_sampler = torch.utils.data.RandomSampler(train_data)
train_set = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=32)
val_sampler = torch.utils.data.RandomSampler(val_data)
val_set = torch.utils.data.DataLoader(val_data, sampler=val_sampler, batch_size=32)

In [84]:
next(iter(train_set))[0].shape

torch.Size([32, 20, 86])

In [100]:
from tqdm import tqdm
epochs = 30
model = LSTMModel(input_size=86, hidden_size=200, num_layers=2, output_size=2, drop_out=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

val_best_acc = 0

for epoch in range(epochs):
    
    model.init_hidden(batch_size=32)
        
    train(model, criterion, optimizer, train_set, epoch)
    val_acc, _ = evaluate(model, criterion, val_set)
        
    if val_acc > val_best_acc:
        val_best_acc = val_acc
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/testmodel.pt")
    if  (epoch - best_epoch) > 10:
        break





0it [00:00, ?it/s]

cpu






1it [00:00,  2.76it/s]

0.0






2it [00:00,  2.82it/s]



3it [00:01,  2.83it/s]



4it [00:01,  2.92it/s]



5it [00:01,  2.89it/s]



6it [00:02,  2.92it/s]



7it [00:02,  2.98it/s]



8it [00:02,  3.05it/s]



9it [00:02,  3.09it/s]



10it [00:03,  3.11it/s]



11it [00:03,  3.07it/s]

...

1.0






892it [04:56,  2.96it/s]



893it [04:56,  2.95it/s]



894it [04:57,  2.99it/s]



895it [04:57,  2.98it/s]



896it [04:57,  3.00it/s]



897it [04:58,  2.93it/s]



898it [04:58,  3.00it/s]



899it [04:58,  3.02it/s]



900it [04:59,  2.97it/s]



901it [04:59,  3.00it/s]

1.0






902it [04:59,  2.98it/s]



903it [05:00,  2.95it/s]



904it [05:00,  2.97it/s]

KeyboardInterrupt: 

In [57]:
model = LSTMModel(input_size=86, hidden_size=200, num_layers=2, output_size=2, drop_out=0.5)

In [16]:
input = npzfile["data"][0:4]

In [58]:
x_input.shape

torch.Size([20, 4, 86])

In [59]:
output = model(x_input)

In [60]:
lstmoutput = output[0]
h, c = output[1]

In [61]:
lstmoutput.shape

torch.Size([20, 4, 200])